## Import Libraries

In [1]:
import os
import json
import pandas as pd
import xgboost as xgb
import dict2xml

already_multiplied = False

### Extract Data

In [2]:
orders = pd.read_csv("data/orders_2015.csv", encoding="latin-1")
order_details = pd.read_csv("data/order_details_2015.csv", encoding="latin-1")
pizza_ingredients = pd.read_csv("data/pizza_types.csv", encoding="latin-1")

In [3]:
ingredients_per_day = pd.read_csv('data/ingredients_per_day_2015.csv', index_col=0)
ingredients_per_day.index = pd.to_datetime(ingredients_per_day.index)
ingredients_per_day = ingredients_per_day.sort_index()
ingredients_per_day.head()

,Alfredo Sauce,Anchovies,Artichoke,Arugula,Asiago Cheese,Bacon,Barbecue Sauce,Barbecued Chicken,Beef Chuck Roast,Blue Cheese,...,Romano Cheese,Sliced Ham,Smoked Gouda Cheese,Soppressata Salami,Spinach,Sun-dried Tomatoes,Thai Sweet Chilli Sauce,Thyme,Tomatoes,Zucchini
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,3,4,43,9,18,24,27,27,8,21,...,21,7,21,3,60,11,22,0,227,7
2015-01-02,2,14,69,9,24,19,19,19,14,18,...,18,21,18,12,61,7,19,0,220,21
2015-01-03,13,11,67,12,30,18,16,16,21,12,...,12,18,12,3,78,19,23,1,195,23
2015-01-04,9,10,49,9,19,11,27,27,6,12,...,12,7,12,0,40,6,13,2,132,8
2015-01-05,19,10,46,8,34,19,5,5,4,18,...,18,9,18,3,54,14,14,2,128,8


## Informe Calidad de Datos

In [4]:
informe = {"orders":{}, "order_details":{}, "pizza_types":{}}

#### orders.csv

In [5]:
informe["orders"]["desciption"] = "csv with every order made in 2015"
informe["orders"]["n_rows"] = orders.shape[0]
informe["orders"]["n_columns"] = orders.shape[1]
informe["orders"]["columns"] = {}
orders.head()

,order_id,date,time
0,1,01/01/2015,11:38:36
1,2,01/01/2015,11:57:40
2,3,01/01/2015,12:12:28
3,4,01/01/2015,12:16:31
4,5,01/01/2015,12:21:30


order_id

In [6]:
informe["orders"]["columns"]["order_id"] = {}
informe["orders"]["columns"]["order_id"]["description"] = "unique id for each order"
informe["orders"]["columns"]["order_id"]["n_nulls"] = orders["order_id"].isnull().sum()
informe["orders"]["columns"]["order_id"]["n_unique"] = orders["order_id"].nunique()

date

In [7]:
informe["orders"]["columns"]["date"] = {}
informe["orders"]["columns"]["date"]["description"] = "date of the order"
informe["orders"]["columns"]["date"]["n_nulls"] = orders["date"].isnull().sum()
informe["orders"]["columns"]["date"]["n_unique"] = orders["date"].nunique()

time

In [8]:
informe["orders"]["columns"]["time"] = {}
informe["orders"]["columns"]["time"]["description"] = "exact time of the order"
informe["orders"]["columns"]["time"]["n_nulls"] = orders["time"].isnull().sum()
informe["orders"]["columns"]["time"]["n_unique"] = orders["time"].nunique()

#### order_details.csv

In [9]:
informe["order_details"]["desciption"] = "csv with information about each order made in 2015"
informe["order_details"]["n_rows"] = order_details.shape[0]
informe["order_details"]["n_columns"] = order_details.shape[1]
informe["order_details"]["columns"] = {}
order_details.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


order_details_id

In [10]:
informe["order_details"]["columns"]["order_details_id"] = {}
informe["order_details"]["columns"]["order_details_id"]["description"] = "unique id for each pizza in each order"
informe["order_details"]["columns"]["order_details_id"]["n_nulls"] = order_details["order_details_id"].isnull().sum()
informe["order_details"]["columns"]["order_details_id"]["n_unique"] = order_details["order_details_id"].nunique()

order_id

In [11]:
informe["order_details"]["columns"]["order_id"] = {}
informe["order_details"]["columns"]["order_id"]["description"] = "unique id for each order"
informe["order_details"]["columns"]["order_id"]["n_nulls"] = order_details["order_id"].isnull().sum()
informe["order_details"]["columns"]["order_id"]["n_unique"] = order_details["order_id"].nunique()

pizza_id

In [12]:
informe["order_details"]["columns"]["pizza_id"] = {}
informe["order_details"]["columns"]["pizza_id"]["description"] = "unique id for each pizza and size"
informe["order_details"]["columns"]["pizza_id"]["n_nulls"] = order_details["pizza_id"].isnull().sum()
informe["order_details"]["columns"]["pizza_id"]["n_unique"] = order_details["pizza_id"].nunique()

quantity

In [13]:
informe["order_details"]["columns"]["quantity"] = {}
informe["order_details"]["columns"]["quantity"]["description"] = "number of pizzas ordered"
informe["order_details"]["columns"]["quantity"]["n_nulls"] = order_details["quantity"].isnull().sum()
informe["order_details"]["columns"]["quantity"]["n_unique"] = order_details["quantity"].nunique()

#### pizza_types.csv

In [14]:
informe["pizza_types"]["desciption"] = "csv with information about each pizza and its ingredients"
informe["pizza_types"]["n_rows"] = pizza_ingredients.shape[0]
informe["pizza_types"]["n_columns"] = pizza_ingredients.shape[1]
informe["pizza_types"]["columns"] = {}
pizza_ingredients.head()

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


pizza_type_id

In [15]:
informe["pizza_types"]["columns"]["pizza_type_id"] = {}
informe["pizza_types"]["columns"]["pizza_type_id"]["description"] = "unique id for each pizza without size"
informe["pizza_types"]["columns"]["pizza_type_id"]["n_nulls"] = pizza_ingredients["pizza_type_id"].isnull().sum()
informe["pizza_types"]["columns"]["pizza_type_id"]["n_unique"] = pizza_ingredients["pizza_type_id"].nunique()

name

In [16]:
informe["pizza_types"]["columns"]["name"] = {}
informe["pizza_types"]["columns"]["name"]["description"] = "full name of the pizza"
informe["pizza_types"]["columns"]["name"]["n_nulls"] = pizza_ingredients["name"].isnull().sum()
informe["pizza_types"]["columns"]["name"]["n_unique"] = pizza_ingredients["name"].nunique()

category

In [17]:
informe["pizza_types"]["columns"]["category"] = {}
informe["pizza_types"]["columns"]["category"]["description"] = "category of the pizza"
informe["pizza_types"]["columns"]["category"]["n_nulls"] = pizza_ingredients["category"].isnull().sum()
informe["pizza_types"]["columns"]["category"]["n_unique"] = pizza_ingredients["category"].nunique()

ingredients

In [18]:
informe["pizza_types"]["columns"]["ingredients"] = {}
informe["pizza_types"]["columns"]["ingredients"]["description"] = "ingredients of the pizza"
informe["pizza_types"]["columns"]["ingredients"]["n_nulls"] = pizza_ingredients["ingredients"].isnull().sum()
informe["pizza_types"]["columns"]["ingredients"]["n_unique"] = pizza_ingredients["ingredients"].nunique()

## Predict

### Load Model

In [19]:
model = xgb.XGBRegressor()
model.load_model('model/ingredients.model')
model_info = json.load(open('model/info.json', 'r', encoding='utf-8'))
model_info["maes"] = pd.Series(model_info["maes"])
model_info["maes"]

Alfredo Sauce               2.694113
Anchovies                   4.073997
Artichoke                  12.581386
Arugula                     3.004441
Asiago Cheese               6.585123
                             ...    
Sun-dried Tomatoes          4.910173
Thai Sweet Chilli Sauce     5.903495
Thyme                       1.188943
Tomatoes                   22.693153
Zucchini                    5.018326
Length: 64, dtype: float64

In [20]:
def mean_ingredients_predict(day,dataframe,previous_days=3,previous_weeks=2,verbose=False):
    '''
    Returns the mean of the ingredients of the previous days and the same day of the previous weeks
    '''
    if verbose:
        print('Calculating mean ingredients to train Day: ',day.date())
    start_day = day-pd.Timedelta(days=previous_days)
    last_day = day-pd.Timedelta(days=1)
    pdays = pd.date_range(start_day,last_day)
    '''Append previous weeks'''
    for w in range(previous_weeks,0,-1):
        date = day-pd.Timedelta(weeks=w)
        pdays = pdays.insert(previous_weeks-w,date)
    '''Fill missing data'''
    for day in pdays:
        if day not in dataframe.index:
            if verbose:
                print('Missing previous day:',day.date())
            delta=day-dataframe.index[0]
            if verbose:
                print("Attempting to predict it")
            if delta.days < previous_weeks*7:
                if verbose:
                    print('Not enough data to predict!')
                    print('Returning mean of previous and next days')
                m=1
                try:
                    return (dataframe.loc[day-pd.Timedelta(days=m)]+dataframe.loc[day+pd.Timedelta(days=m)])/(2*m)
                except:
                    if verbose:
                        print('To enought data to calculate mean')
                        print("Returning mean of all data")
                    return dataframe.mean()
            dataframe.loc[day] = predict(day,dataframe,verbose=verbose)
    return dataframe.loc[pdays].mean()

def predict(day,dataframe,verbose=False):
    '''
    Predict the ingredients for the given day
    Inputs:
        day: datetime object
        verbose: bool
        dataframe: dataframe with the ingredients
    Outputs:
        ingredients: dataframe with the ingredients
    '''
    p_days = model_info["previous_days"]
    p_weeks = model_info["previous_weeks"]
    mean = mean_ingredients_predict(day,verbose=verbose,dataframe=dataframe,previous_days=p_days,previous_weeks=p_weeks)
    dayofweek = day.dayofweek
    month = day.month
    X = pd.DataFrame([mean],columns=mean.index)
    X['dayofweek'] = dayofweek
    X['month'] = month
    pred = model.predict(X).tolist()[0]
    return pd.Series(pred,index=mean.index)

def predict_stock(monday,dataframe,margins=None,verbose=False):
    '''
    Predict the stock for the given week, summing up the ingredients for each day
    Returns: DataFrame with the predicted stock for each ingredient
    '''
    dataframe = dataframe.copy()
    if margins is None:
        margins = 0
    stock = predict(monday,verbose=verbose,dataframe=dataframe)+margins
    for day in pd.date_range(monday+pd.Timedelta(days=1),monday+pd.Timedelta(days=6)):
        stock += predict(day,verbose=verbose,dataframe=dataframe)+margins
    return stock

def real_stock(monday,dataframe=None):
    '''
    Returns the real stock for the given week
    '''
    stock = dataframe.loc[monday].copy()
    for day in pd.date_range(monday+pd.Timedelta(days=1),monday+pd.Timedelta(days=6)):
        stock += dataframe.loc[day]
    return stock

In [21]:
available_data = ingredients_per_day.loc[:'2015-06-05']
monday = pd.to_datetime('2015-06-08')
stock = predict_stock(monday,available_data,margins=model_info["maes"],verbose=False)
stock

Alfredo Sauce                59.951117
Anchovies                    96.969573
Artichoke                   426.717342
Arugula                      90.776775
Asiago Cheese               193.685369
                              ...     
Sun-dried Tomatoes          110.463101
Thai Sweet Chilli Sauce     160.302032
Thyme                        16.729564
Tomatoes                   1354.472246
Zucchini                    134.097570
Length: 64, dtype: float64

In [22]:
prediction = {"info":{}}
prediction["info"]["prediction_week"] = monday.date().isoformat()
prediction["info"]["last_data_available"] = available_data.index[-1].date().isoformat()
prediction["predicted_ingredients"] = {}
for ingredient in stock.index:
    prediction["predicted_ingredients"][ingredient] = int(stock[ingredient])

In [23]:
report={
    "data_quality_report":informe,
    "prediction":prediction
}

### Save report in xml

In [24]:
xml = dict2xml.dict2xml(report,wrap="prediction",indent="    ")
if not os.path.exists("output"):
    os.mkdir("output")
with open('output/report_2015.xml', 'w', encoding='utf-8') as f:
    f.write(xml)